# Generación de texto con una red recurrente

* El ejemplo ocupa texto de Shakespeare
* Podríamos repetir el proceso con Cervantes?

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

2023-10-25 10:02:59.027077: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
ruta_al_archivo = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [3]:
texto = open(ruta_al_archivo, 'rb').read().decode(encoding='utf-8')

In [4]:
print(f'Longitud del texto: {len(texto)} carácteres')

Longitud del texto: 1115394 carácteres


In [5]:
print(texto[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
vocab = sorted(set(texto))

In [7]:
print(f'{len(vocab)} carácteres únicos')

65 carácteres únicos


## Vectorización del texto

Antes de entrenar el modelo, necesitamos convertir los *strings* a una representación numérica.

La capa `tf.keras.layers.StringLookup` puede convertir cada carácter en un ID numérico. Necesita que el texto esté separado en fichas (*tokens*) primero.

In [8]:
ejemplos = ['abcdefg', 'xyz']

caracteres = tf.strings.unicode_split(ejemplos, input_encoding='UTF-8')

2023-10-25 10:03:11.588264: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [9]:
caracteres

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [10]:
ids_de_cars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)

In [11]:
ids = ids_de_cars(caracteres)

In [12]:
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

Podemos invertir la representación numérica para extraer carácteres de nuevo. (Usamos un método `get_vocabulary` de la capa `StringLookup` para obtener el vocabulario).

In [13]:
cars_de_ids = tf.keras.layers.StringLookup(vocabulary=ids_de_cars.get_vocabulary(), invert=True, mask_token=None)

In [14]:
cars = cars_de_ids(ids)

In [15]:
cars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [16]:
tf.strings.reduce_join(cars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [17]:
def texto_de_ids(ids):
    return tf.strings.reduce_join(cars_de_ids(ids), axis=-1)

### La tarea de predicción

Dado un carácter, o una secuencia de carácteres, cuál carácter es el más probable que viene después? Vamos a entrenar el modelo para resolver este problema.

La entrada es una secuencia de carácteres (en su representación numérica) y la predicción será el próximo carácter en cada paso del tiempo.

Las redes recurrentes mantienen un estado interno que depende de los elementos vistos previamente. Así que, la pregunta es: dado todos los carácteres obtenidos hasta este momento, cuál es el próximo?

### Crear ejemplos de entrenamiento y objetivos

Dividimos el texto en secuencias de ejemplo. Cada secuencia de entrada contendrá `longitud_sec` carácteres del texto.

Para cada secuencia de entrada, los objetivos que corresponden contienen el mismo número de carácteres, pero desplazado un carácter a la derecha.

Rompimos el texto en pedazos de `longitud_sec+1`. Por ejemplo, digamos que `longitud_sec` = 4, y nuestro texto es "Hello". La secuencia de entrada sería "Hell" y la secuencia de objetivo sería "ello".

Ocupamos la función `tf.data.Dataset.from_tensor_slices` para convertir el vector de texto en una secuencia de indices de carácteres.

In [18]:
todos_ids = ids_de_cars(tf.strings.unicode_split(texto, 'UTF-8'))

In [19]:
todos_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [20]:
ids_dataset = tf.data.Dataset.from_tensor_slices(todos_ids)

In [21]:
ids_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>

In [22]:
for ids in ids_dataset.take(10):
    print(cars_de_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


2023-10-25 10:03:53.697317: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [1115394]
	 [[{{node Placeholder/_0}}]]


In [23]:
longitud_sec = 100

El método `batch` facilita la conversión de carácteres individuales a secuencias del tamaño requerido.

In [24]:
secuencias = ids_dataset.batch(longitud_sec+1, drop_remainder=True)

In [25]:
for sec in secuencias.take(1):
    print(cars_de_ids(sec))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


2023-10-25 10:03:59.249081: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [1115394]
	 [[{{node Placeholder/_0}}]]


In [26]:
for sec in secuencias.take(5):
    print(texto_de_ids(sec).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


2023-10-25 10:04:03.810549: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [1115394]
	 [[{{node Placeholder/_0}}]]


Para el entrenamiento necesitamos un dataset de pares de `(entrada, etiqueta)`, donde `entrada` y `etiqueta` son secuencias. En cada paso del tiempo la entrada es el carácter actual y la etiqueta es el carácter subsiguiente.

In [27]:
def dividir_entrada_objetivo(secuencia):
    texto_entrada = secuencia[:-1]
    texto_objetivo = secuencia[1:]
    return texto_entrada, texto_objetivo

In [28]:
dividir_entrada_objetivo(list("TensorFlow"))

(['T', 'e', 'n', 's', 'o', 'r', 'F', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'F', 'l', 'o', 'w'])

In [29]:
dataset = secuencias.map(dividir_entrada_objetivo)

In [30]:
dataset

<_MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int64, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>

In [31]:
for ejemplo_entrada, ejemplo_objetivo in dataset.take(1):
    print("Entrada: ", texto_de_ids(ejemplo_entrada).numpy())
    print("Objetivo: ", texto_de_ids(ejemplo_objetivo).numpy())

Entrada:  b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Objetivo:  b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


2023-10-25 10:04:16.748342: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [1115394]
	 [[{{node Placeholder/_0}}]]


### Crear lotes de entrenamiento

Tenemos que barajar los datos y empacarlos en lotes.

In [32]:
# Tamaño de los lotes
LOTE = 64

# Tamaño del "buffer" para barajar los datos
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(LOTE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

In [33]:
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Construir el modelo

Vamos a usar el API de "subclassing" (en vez del API funcional que usabamos en la parte sobre redes de *feed-forward*).

El modelo tiene 3 capas:

* `tf.keras.layers.Embedding`: la capa de entrada. Una tabla de tipo "lookup" (entrenable) que va a mapear cada carácter-ID a un vector con dimensiones `embedding_dim`.
<p> <br> </p>
* `tf.keras.layers.GRU`: un tipo de RNN con tamaño `units=rnn_units` (también podríamos usar LSTM aquí)
<p> <br> </p>
* `tf.keras.layers.Dense`: la capa de salida, con `vocab_size` salidas. Produce un *logit* por cada carácter del vocabulario. Estos son los *log-likelihood* de cada carácter según el modelo.

In [34]:
vocab_size = len(ids_de_cars.get_vocabulary())

embedding_dim = 256

rnn_units = 1024

In [35]:
class MiModelo(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)
        
    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)
        
        if return_state:
            return x, states
        else:
            return x

In [36]:
modelo = MiModelo(vocab_size=vocab_size, 
                  embedding_dim=embedding_dim, 
                  rnn_units=rnn_units)

Un diagrama del modelo:

![](text_generation_training.png)

Primero, verificamos la forma de la salida

In [37]:
for lote_entrada_ejemplo, lote_objetivo_ejemplo in dataset.take(1):
    lote_predicciones_ejemplo = modelo(lote_entrada_ejemplo)
    print(lote_predicciones_ejemplo.shape, "# (lote, secuencia, vocabulario)")

2023-10-25 10:04:32.704102: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [1115394]
	 [[{{node Placeholder/_0}}]]
2023-10-25 10:04:32.704784: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [1115394]
	 [[{{node Placeholder/_0}}]]


(64, 100, 66) # (lote, secuencia, vocabulario)


In [38]:
modelo.summary()

Model: "mi_modelo"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


Para obtener predicciones del modelo, tenemos que extraer muestras de la distribución de salidas. Esta distribución se define por los *logits* sobre el vocabulario de carácteres.

Si ocupamos simplemente `argmax` el modelo se puede congelar en un *loop* infinito.

In [39]:
indices_muestra = tf.random.categorical(lote_predicciones_ejemplo[0], num_samples=1)

In [40]:
indices_muestra = tf.squeeze(indices_muestra, axis=-1).numpy()

In [41]:
indices_muestra

array([30, 20, 14, 49, 16, 33,  8, 56, 62, 55, 43, 33,  9, 37, 59, 21, 62,
        5, 43,  9, 23, 39, 62, 11, 18, 25, 16, 47, 43, 65, 25, 27, 40, 59,
       27, 56, 34,  1,  4, 62, 59, 48, 42, 14, 60, 38, 44,  3, 43, 45, 33,
       18, 25,  6, 27, 49, 35, 65,  4, 30, 25,  5,  5, 49, 44, 54,  1, 19,
       49,  7, 60, 44, 45, 58, 10,  7, 62, 57, 59, 33, 45, 20, 63, 23, 13,
       41, 64, 62, 10, 56, 18, 59, 34, 20, 15,  3, 33, 53, 24, 17])

In [42]:
print("Entrada:\n", texto_de_ids(lote_entrada_ejemplo[0]).numpy())

Entrada:
 b"ngerous unsafe lunes i' the king,\nbeshrew them!\nHe must be told on't, and he shall: the office\nBecom"


In [43]:
print("Predicciones próximo carácter:\n", texto_de_ids(indices_muestra).numpy())

Predicciones próximo carácter:
 b"QGAjCT-qwpdT.XtHw&d.JZw:ELChdzLNatNqU\n$wticAuYe!dfTEL'NjVz$QL&&jeo\nFj,uefs3,wrtTfGxJ?byw3qEtUGB!TnKD"


## Entrenar el modelo

Ahora tenemos un problema estandar de clasificación. Dado el estado previo del RNN, y la entrada en este paso, predice la clase del próximo carácter.

### Optimizador y función de pérdida

Podemos usar `tf.keras.losses.sparse_categorical_crossentropy` ya que se aplica en la última dimensión de las predicciones.

Ya que el modelo retorna *logits*, hay que especificar `from_logits=True`.

In [44]:
perdida = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [45]:
ejemplo_lote_perdida_promedia = perdida(lote_objetivo_ejemplo, lote_predicciones_ejemplo)

In [46]:
print("Forma predicción: ", lote_predicciones_ejemplo.shape, "# (lote, secuencia, vocabulario)")

Forma predicción:  (64, 100, 66) # (lote, secuencia, vocabulario)


In [47]:
print("Perdida promedia: ", ejemplo_lote_perdida_promedia)

Perdida promedia:  tf.Tensor(4.190913, shape=(), dtype=float32)


Un modelo recién inicializado no debería estar demasiado seguro de sus salidas. Podemos confirmar eso usando la exponencial de la perdida promedia, y verificando que es aproximadamente igual al tamaño del vocabulario. Una perdida mucho más grande indica que el modelo es muy seguro de sus respuestas incorrectas, y su inicialización es mala.

In [48]:
tf.exp(ejemplo_lote_perdida_promedia).numpy()

66.08311

Compilamos el modelo (basicamente creando los grafos etc.)

In [49]:
modelo.compile(optimizer='adam', loss=perdida)

### Checkpoints

Usamos *checkpoints* para guardar los parámetros durante el entrenamiento, por si acaso...

In [50]:
checkpoint_dir = './training_checkpoints'

In [51]:
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

In [52]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

## Entrenar!

Entrenamos el modelo para $10$ epocas, para tener un tiempo razonable... También podríamos usar GPU para acelerar el entrenamiento.

In [53]:
EPOCAS = 10

In [54]:
historia = modelo.fit(dataset, epochs=EPOCAS, callbacks=[checkpoint_callback])

Epoch 1/10


2023-10-25 10:05:23.870207: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [1115394]
	 [[{{node Placeholder/_0}}]]
2023-10-25 10:05:23.870634: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [1115394]
	 [[{{node Placeholder/_0}}]]
2023-10-25 10:05:24.195407: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_d

172/172 [==============================] - 521s 3s/step - loss: 2.6929
Epoch 2/10
172/172 [==============================] - 455s 3s/step - loss: 1.9688
Epoch 3/10
172/172 [==============================] - 444s 3s/step - loss: 1.6921
Epoch 4/10
172/172 [==============================] - 441s 3s/step - loss: 1.5349
Epoch 5/10
172/172 [==============================] - 448s 3s/step - loss: 1.4392
Epoch 6/10
172/172 [==============================] - 435s 3s/step - loss: 1.3732
Epoch 7/10
172/172 [==============================] - 454s 3s/step - loss: 1.3216
Epoch 8/10
172/172 [==============================] - 454s 3s/step - loss: 1.2768
Epoch 9/10
172/172 [==============================] - 448s 3s/step - loss: 1.2365
Epoch 10/10
172/172 [==============================] - 448s 3s/step - loss: 1.1965


### Generar texto

Podemos generar texto con el modelo usando un ciclo, con seguimiento del estado interno.

![](text_generation_sampling.png)

Cada vez que llamamos al modelo, pasamos texto y su estado interno. El modelo devuelve una predicción para el próximo carácter y su estado nuevo. Pasamos la predicción y el estado de nuevo al modelo para generar más texto.

In [55]:
class UnPaso(tf.keras.Model):
    def __init__(self, modelo, cars_de_ids, ids_de_cars, temperatura=1.0):
        super().__init__()
        self.temperatura = temperatura
        self.modelo = modelo
        self.cars_de_ids = cars_de_ids
        self.ids_de_cars = ids_de_cars
        
        # Crear una máscara para prevenir generación de "[UNK]"
        #saltar_ids = self.ids_de_cars('[UNK]').numpy()
        #mascara_escasa = tf.SparseTensor(
            # Ponemos -inf en cada indice malo
        #    values=-float('inf'),
        #    indices=saltar_ids,
            # igualar la forma al vocabulario
        #    dense_shape=[len(ids_de_cars.get_vocabulary())])
        #self.mascara_prediccion = tf.sparse.to_dense(mascara_escasa)
        
    @tf.function
    def generar_un_paso(self, entradas, estados=None):
        # Convertir strings a IDs de fichas
        entrada_cars = tf.strings.unicode_split(entradas, 'UTF-8')
        entrada_ids = self.ids_de_cars(entrada_cars).to_tensor()
        
        # Ejecutar el modelo
        # logits_predichos.shape es [lote, carácteres, logits_próximo_carácter]
        logits_predichos, estados = self.modelo(inputs=entrada_ids,
                                                states=estados,
                                                return_state=True)
        # Ocupar solamente la última predicción
        logits_predichos = logits_predichos[:, -1, :]
        logits_predichos = logits_predichos/self.temperatura
        # Aplicar la mascara de predicción para prevenir "[UNK]"
        logits_predichos = logits_predichos #+ self.mascara_prediccion
        
        # Obtener muestra de los logits de salida para generar fichas de IDs
        ids_predichas = tf.random.categorical(logits_predichos, num_samples=1)
        ids_predichas = tf.squeeze(ids_predichas, axis=-1)
        
        # Convertir de fichas de ID a carácteres
        cars_predichos = self.cars_de_ids(ids_predichas)
        
        # Retornar los carácteres y el estado del modelo
        return cars_predichos, estados

In [56]:
un_paso_modelo = UnPaso(modelo, cars_de_ids, ids_de_cars)

In [57]:
comienzo = time.time()
estados = None
proximo_car = tf.constant(['ROMEO:'])
resultado = [proximo_car]

for n in range(1000):
    proximo_car, estados = un_paso_modelo.generar_un_paso(proximo_car, estados=estados)
    resultado.append(proximo_car)
    
resultado = tf.strings.join(resultado)
final = time.time()

2023-10-26 13:55:22.873876: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-26 13:55:22.874897: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-26 13:55:22.875975: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [58]:
print(resultado[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nTiempo de ejecución:', final - comienzo)

ROMEO:
That Roomey' but thy profit of him
And city of him, put every hand.

QUEEN MARGARET:
Who had he shall not.
Mard that the mercy should but here appear
At palm and no't:
See, all post gagerel, his country may
In the sallignhest sent to-day.
Within my spirit of a city out on his paples,
Even so in the valians. Take me
The one poor cliff may pray thee. Bod!

MERENIUS:
You have no out my lord;
So hath made jow in any tame the sun.
Alto some other in done, thy heart will tear,
Dreamnedly shall 't. He going presently rescoiled him;
And now he knows my reammission which our corvins
That I may chatte; and do good pubull
And seen to long wething hours made
A gentleman in our own sovereing stand:
Being sawns uppell'd, and so we must do
And bring the sweetest squaring he hads aponet;
And so it is right old Mariqay be you, till King Henry's right,
Have dancing should to action in deft he usurp.

KING RICHARD II:
Go, A wife in tume of Ronoraft she!
The contentios of the Duke of Plater natural

Se puede implementar el paso de entrenamiento directamente, usando `tf.GradientTape`.

In [103]:
class CustomTraining(MiModelo):
    @tf.function
    def train_step(self, inputs):
        inputs, labels = inputs
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = self.loss(labels, predictions)
        grads = tape.gradient(loss, modelo.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, modelo.trainable_variables))

        return {'loss': loss}

In [104]:
modelo = CustomTraining(vocab_size=len(ids_de_cars.get_vocabulary()),
                        embedding_dim=embedding_dim,
                        rnn_units=rnn_units)

In [105]:
modelo.compile(optimizer = tf.keras.optimizers.Adam(),
               loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [106]:
modelo.fit(dataset, epochs=1)

2023-10-17 15:41:13.429748: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-17 15:41:13.431293: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-17 15:41:13.432282: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

  6/172 [>.............................] - ETA: 7:54 - loss: 4.2275

KeyboardInterrupt: 